In [ ]:
import ContentLibrary as cl
from pdfminer.high_level import extract_text

library = cl.ContentLibrary('/Users/paul/Desktop/FOM_MSc_Thesis.bib')


In [ ]:
class Document():

    def __init__(self, entry):
        base_path = '/Users/paul/Zotero/storage/' 
        self.entry = entry
        self.title = self.entry.fields['title']
        self.fields = self.entry.fields.keys()
        self.is_valid = False
        if 'file' in self.fields:
           self.file = self.entry.fields['file'].split(base_path)[1].split(':')[0]
        else:
            self.file = ''
            self.text = None
        self.is_pdf = bool(re.search('.pdf', self.file))
        if self.is_pdf:
            self.text = extract_text(base_path + self.file)
            self.is_valid = True
